In [279]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
# import cv2
import numpy as np 
import pandas as pd
import re
import urllib
import pickle
import query
import re
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from fuzzywuzzy import fuzz

%load_ext autoreload
%autoreload 2
%aimport query

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [309]:
import random

def get_data_from_page(rows,games_dict):
    game_d = {}
    for row in rows:
        for lab in ['Name','First release date','Platfrom',
                    'Developer','Publisher','Genre','Theme',
                    'Franchise','Aliasses']:
            if lab in row.text:
                if lab == 'First release date':
                    game_d[lab] = row.find('td').find('span').text
                elif lab == 'Genre' or lab == 'Platfrom':
                    if len(row.find('td').findAll('a')) > 1:
                        game_d[lab] = [i.text for i in row.find('td').findAll('a')]
                    elif row.find('td').find('a') == None:
                        game_d[lab] = None
                    else:
                        game_d[lab] = row.find('td').find('a').text    
                elif lab == 'Publisher':
                    if len(row.find('td').findAll('a')) > 1:
                        game_d['Publisher_extra'] = [i.text for i in row.find('td').findAll('a')]
                        game_d[lab] = row.find('td').find('a').text
                    elif row.find('td').find('a') == None:
                        game_d[lab] = None
                    else:
                        game_d[lab] = row.find('td').find('a').text
                else:
                    try:
                        game_d[lab] = row.find('td').find('a').text
                    except:
                        game_d[lab] = None
    return game_d

def get_game_details(title,driver,games_dict):
    t = title.split('(')[0]
    time.sleep(random.randint(0,2))
#     driver.find_element_by_partial_link_text(t).click()
    search_results = driver.find_elements_by_class_name('media')
    search_results[0].click()
    time.sleep(random.randint(0,2))
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    rows = soup.findAll('tr')
    games_dict[title] = get_data_from_page(rows,games_dict)
    return 'done'

def get_game_details_2(title,driver,games_dict):
    time.sleep(random.randint(0,2))
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    rows = soup.findAll('tr')
    games_dict[title] = get_data_from_page(rows,games_dict)
    return 'done'

In [6]:
DB_CONNECTION = query.get_db_connection('../../../.supreme_db_creds.pkl','Sandbox_Elena_Morais',dbhost='centaur')

In [7]:
all_tables = {}

for region in ['USA','EU']:
    for yr in range(2013,2021):
        title = f'{region}_VGT_{yr}_Table_update'
        all_tables[title] = query.return_table(DB_CONNECTION,'Sandbox_Elena_Morais',title)

In [11]:
tabs = []
for tab in all_tables.keys():
    tabs.append(all_tables[tab][['title','publisher','franchise_update']])

In [12]:
full_table = pd.concat(tabs)

In [18]:
titles = list(set(full_table.title.to_list()))

In [139]:
def save_pickle(file_name,obj):
    with open(file_name, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return file_name + ' is saved'

def open_pickle(file_name):
    with open(file_name, 'rb') as handle:
        obj = pickle.load(handle)
    return obj

In [140]:
save_pickle('set_of_titles.pkl',titles)

'set_of_titles.pkl is saved'

In [95]:
# options = webdriver.ChromeOptions()
# options.add_argument("start-maximized");
# options.add_argument("disable-infobars")
# options.add_argument("--disable-extensions")
# chrome_options=options,

In [145]:
html_ex = driver.page_source
html_ex

'<html lang="en" class="js no-touch  blobconstructor blob-constructor cors dataview history filereader localstorage sessionstorage canvas canvastext video todataurljpeg todataurlpng todataurlwebp cssgradients fileinput srcset supports no-touchevents formvalidation fullscreen cssanimations backgroundsize csscolumns csscolumns-width csscolumns-span csscolumns-fill csscolumns-gap csscolumns-rule csscolumns-rulecolor csscolumns-rulestyle csscolumns-rulewidth csscolumns-breakbefore csscolumns-breakafter csscolumns-breakinside no-cssgridlegacy cssgrid flexbox flexboxlegacy cssmask csstransforms csstransforms3d backgroundcliptext exiforientation" data-useragent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" data-platform="MacIntel" data-triggered="true"><head><script src="https://www.googletagservices.com/activeview/js/current/osd.js?cb=%2Fr20100101"></script><script type="text/javascript" async="async" src="https://s

In [200]:
driver = webdriver.Chrome(executable_path='/Users/elena.morais/Documents/chromedriver_2') 
driver.get('https://www.giantbomb.com')

time.sleep(30)

In [262]:
c = 0 
for title in titles[1978:]:
    print(title)

    time.sleep(random.randint(2,5))
    driver.find_elements_by_xpath('//div[@class="js-masthead-toggle masthead-toggle masthead-toggle-search"]')[1].click()
    time.sleep(random.randint(0,2))
    driver.find_elements_by_xpath("//option[@value='game']")[0].click()
    driver.find_element_by_xpath('//input[@class="js-site-search-query"]').send_keys(title)
    time.sleep(random.randint(0,2))
#     driver.find_element_by_xpath('//select[@class="dropdown--selector"]').click()
#     driver.find_element_by_xpath('//option[@value="game"]').click()
    time.sleep(random.randint(0,2))
    driver.find_element_by_xpath('//input[@class="js-site-search-query"]').send_keys(u'\ue007')
#     river-search__select
    get_game_details(title,driver,games_dict)
    
    if c%20 == 0:
        save_pickle('giant_bomb_updated_game_info.pkl',games_dict)
        

save_pickle('giant_bomb_updated_game_info.pkl',games_dict)
        

Pokemon Sword: Expansion Pass (The Isle of Armor / The Crown Tundra)
Don Bradman Cricket 14
Hyperdimension Neptunia: Re;Birth3 - V Generati
Danganronpa Another Episode: Ultra Despair Girl
Adventure Time: The Secret of the Nameless Kingdom
Elite: Dangerous
Shin Megami Tensei: Strange Journey Redux
FIFA Soccer 13
Ni no Kuni: Wrath of the White Witch for Nintendo Switch
Payday 3
Marvel's Avengers
Soul Sacrifice
Steins;Gate 0
Chaos;Child
Nike+ Kinect Training
F1 2020
Adventure Time: Pirates of the Enchiridion
Penny Punching Princess
Astroneer
Crusader Kings III
PixelJunk Monsters 2
Knowledge is Power
Rise of the Guardians: The Video Game
Port Royale 4
Bladestorm: Nightmare
BlazBlue: Chrono Phantasma Extend
The Trash Pack
The Legend of Zelda: Tri Force Heroes
Buildings Have Feelings Too
Puzzles & Dragons Z + Puzzles & Dragons Super Mario Bros. Edition
Disney Magical World 2
Batman: A Telltale Games Series
NHL 17
Nights of Azure 2: Bride of the New Moon
Dr. Kawashima's Devilish Brain Trainin

In [267]:
df = pd.DataFrame.from_dict(games_dict, orient='index')

In [276]:
df = df.reset_index().rename(columns={'index':'vgt_name'})

In [270]:
df.Publisher.isnull().sum()

31

In [277]:
def get_fuzzy_match(row):
    name = row['vgt_name']
    name1 = row['Name']
    return fuzz.token_set_ratio(name, name1)

In [273]:
df.index

Index(['Dragon Quest XI S: Echoes of an Elusive Age - D', 'TANK! TANK! TANK!',
       'Road Rage', 'Knack 2', 'Destiny: The Taken King', 'The Banner Saga 2',
       'The Witch and the Hundred Knight', 'Code of Princess',
       'Hatsune Miku: Project Mirai DX', 'Code: Realize - Guardian of Rebirth',
       ...
       'Harvest Moon: A New Beginning', 'Grounded', 'Pro Cycling Manager 2015',
       'Atelier Ryza: Ever Darkness & the Secret Hideout', 'Starfield',
       'Dark Rose Valkyrie', 'Dead Space 3', 'Olympic Games Tokyo 2020',
       'Windbound', 'Titanfall 2'],
      dtype='object', length=2030)

In [280]:
df['fuzzy_ratio'] = df.apply(get_fuzzy_match, axis=1)

In [311]:
# df.sort_values("fuzzy_ratio", ascending = True)
df[df['fuzzy_ratio'] < 90].iloc[55:,:]

### hard code updates:

'Disgaea 5 Complete':

Happy Birthdays : https://www.nintendo.com/games/detail/happy-birthdays-switch/
Inazuma Eleven 3: Team Ogre Attacks: https://www.metacritic.com/game/3ds/inazuma-eleven-3-team-ogre-attacks!/details
'Sherlock Holmes and the Mystery of the Frozen C''Sherlock Holmes and the Mystery of the Frozen City':https://www.metacritic.com/game/3ds/sherlock-holmes-and-the-mystery-of-the-frozen-city
Imagine: Fashion Life: https://www.nintendo.com/games/detail/imagine-fashion-life-3ds/
The Elder Scrolls Online: Murkmire: https://elderscrolls.fandom.com/wiki/The_Elder_Scrolls_Online:_Murkmire,
The Elder Scrolls Online: Summerset: https://elderscrolls.fandom.com/wiki/The_Elder_Scrolls_Online:_Summerset
The Elder Scrolls Online: Greymoor: https://store.steampowered.com/app/1205430/The_Elder_Scrolls_Online__Greymoor/
Resident Evil: Resistance:
Inazuma Eleven 3: Bomb Blast: https://www.metacritic.com/game/3ds/inazuma-eleven-3-bomb-blast
Inazuma Eleven 3: Lightning Bolt: 
New Style Boutique: https://www.giantbomb.com/style-savvy-trendsetters/3030-39284/
New Style Boutique 3: Styling Star: https://www.giantbomb.com/style-savvy-styling-star/3030-61367/
Attack on Titan: Wings of Freedom: https://www.giantbomb.com/attack-on-titan/3030-50516/
Jewel Quest: The Sapphire Dragon: https://nintendo.fandom.com/wiki/Jewel_Quest:_The_Sapphire_Dragon
Hitman: Definitive Edition: https://www.metacritic.com/game/playstation-4/hitman-definitive-edition/details












Mechs vs. Minions: https://leagueoflegends.fandom.com/wiki/Mechs_vs._Minions
European Ship Simulator: https://store.steampowered.com/app/299250/European_Ship_Simulator/
Puzzler: Brain Games: https://www.nintendolife.com/reviews/3ds/puzzler_brain_games
Payday 3: https://www.ign.com/games/payday-3
Dr. Kawashima's Devilish Brain Training: Can You Stay Focused: https://www.nintendo.com.au/catalogue/dr-kawashimas-devilish-brain-training-can-you-stay-focused


In [ ]:
hand_done_titles = {'Happy Birthdays':{'Name':'Happy Birthdays','First release date':'Jun 05, 2018',
                    'Platfrom':'Switch','Developer':['Arc System Works', 'TOYBOX Inc.'],
                    'Publisher':'NIS America Inc.','Genre':'Simulation','Theme':None,
                    'Franchise':None,'Aliasses':None},
'Inazuma Eleven 3: Team Ogre Attacks':{'Name':'Inazuma Eleven 3: Team Ogre Attacks!',
                                       'First release date':'Feb 14, 2014',
                                        'Platfrom':'3DS,DS','Developer':'Level 5',
                                        'Publisher':'Level 5','Genre':['Sports', 'Traditional', 
                                                                        'Soccer', 'Arcade'],
                                       'Theme':None,'Franchise':'Inazuma Eleven 3','Aliasses':None},
'Sherlock Holmes and the Mystery of the Frozen C':{'Name':'Sherlock Holmes and the Mystery of the Frozen City',
                                                   'First release date':'Q2 2012','Platfrom':'3DS',
                                                   'Developer':'Frogwares','Publisher':'Focus Home Interactive',
                                                   'Genre':['Action Adventure', 'Fantasy'],'Theme':None,
                                                   'Franchise':'Sherlock Holmes','Aliasses':None},
 'Sherlock Holmes and the Mystery of the Frozen City':{'Name':'Sherlock Holmes and the Mystery of the Frozen City',
                                                       'First release date':'Q2 2012','Platfrom':'3DS',
                                                       'Developer':'Frogwares','Publisher':'Focus Home Interactive',
                                                       'Genre':['Action Adventure', 'Fantasy'],'Theme':None,
                                                       'Franchise':'Sherlock Holmes','Aliasses':None},
 'Imagine: Fashion Life':{'Name':'Imagine Fashion Life','First release date':'May 8, 2012',
                          'Platfrom':['Nintendo DS', 'Nintendo 3DS', 'Wii', 'Microsoft Windows'],
                          'Developer':None,'Publisher':'Ubisoft','Genre':'Simulation','Theme':'Fsshion',
                          'Franchise':'Imagine','Aliasses':None},
  'The Elder Scrolls Online: Greymoor':{'Name':'The Elder Scrolls Online - Greymoor',
                                        'First release date':'May 26, 2020',
                                        'Platfrom':['PC','PlayStation 4','Xbox One'],'Developer':'Zenimax Online Studios',
                                        'Publisher':'Bethesda Softworks','Genre':['Action','RPG'],'Theme':None,
                                        'Franchise':'The Elder Scrolls Online','Aliasses':},
  'The Elder Scrolls Online: Murkmire':{'Name':'','First release date':'October 22, 2018',
                    'Platfrom':['PC','PlayStation 4','Xbox One'],'Developer':'Zenimax Online Studios',
                    'Publisher':'Bethesda Softworks','Genre':['Action','RPG'],'Theme':None,
                    'Franchise':'The Elder Scrolls Online','Aliasses':None},
  'The Elder Scrolls Online: Summerset':{'Name':'','First release date':'April 16th, 2018',
                    'Platfrom':['PC','PlayStation 4','Xbox One'],'Developer':'Zenimax Online Studios',
                    'Publisher':'Bethesda Softworks','Genre':['Action','RPG'],'Theme':None,
                    'Franchise':'The Elder Scrolls Online','Aliasses':None},
  'Resident Evil: Resistance':{'Name':'Resident Evil Resistance','First release date':None,
                               'Platfrom':'PC','Developer':['CAPCOM Co., Ltd.', 'NeoBards Entertainment Ltd.'],
                               'Publisher':'CAPCOM Co., Ltd.','Genre':'Action','Theme':'Zombies',
                               'Franchise':'Resident Evil','Aliasses':None},
  'Inazuma Eleven 3: Bomb Blast':{'Name':'Inazuma Eleven 3: Bomb Blast','First release date':'Sep 27, 2013',
                               'Platfrom':'3DS,DS','Developer':'Level 5',
                               'Publisher':'Level 5','Genre':'Sports, RPG, Soccer, Arcade','Theme':None,
                               'Franchise':'Inazuma Eleven 3','Aliasses':None},
 'Inazuma Eleven 3: Lightning Bolt':{'Name':'Inazuma Eleven 3: Lightning Bolt','First release date':'Jul 01, 2010',
                               'Platfrom':'3DS,DS','Developer':'Level 5',
                               'Publisher':'Level 5','Genre':'Sports, RPG, Soccer, Arcade','Theme':None,
                               'Franchise':'Inazuma Eleven 3','Aliasses':None},
 'New Style Boutique':{'Name':'Style Savvy: Trendsetters','First release date':'September 27, 2012',
                               'Platfrom':'3DS','Developer':'syn Sophia,Nintendo SPD',
                               'Publisher':'Nintendo','Genre':'Simulation','Theme':'fashion,Management',
                               'Franchise':'New Style Boutique',
                       'Aliasses':'Wagamama Fashion: Girls Mode – Greed Declaration,Style Savvy 3DS,Nintendo presents: New Style Boutique,Girls Style: Yokubari Sengen!'},
 'New Style Boutique 3: Styling Star':{'Name':'Style Savvy: Styling Star',
                                       'First release date':'November 2, 2017',
                                       'Platfrom':'3DS','Developer':'syn Sophia',
                                       'Publisher':'Nintendo','Genre':'Simulation','Theme':'fashion,Management',
                                       'Franchise':'New Style Boutique',
                                       'Aliasses':'Girls Mode 4 Star ☆ Stylist,Girls Mode 4 スター☆スタイリスト,Style Savvy 4,Nintendo presents: New Style Boutique 3 - Styling Star'},
 'Attack on Titan: Wings of Freedom':{'Name':'Attack on Titan',
                                      'First release date':'February 18, 2016',
                               'Platfrom':'PlayStation 3,PlayStation Network (PS3),PC,PlayStation Vita,PlayStation Network (Vita),Xbox One,PlayStation 4',
                                      'Developer':'Omega Force',
                               'Publisher':'KOEI TECMO GAMES CO., LTD.','Genre':'Action','Theme':'Anime,Horror',
                               'Franchise':'Attack on Titan','Aliasses':'A.O.T. Wings of Freedom'},
  'Jewel Quest: The Sapphire Dragon':{'Name':'Jewel Quest: The Sapphire Dragon','First release date':'10/20/2011',
                               'Platfrom':'PC','Developer':'iWin',
                               'Publisher':'iWin','Genre':'puzzle','Theme':None,
                               'Franchise':'Jewel Quest','Aliasses':None},
  'Hitman: Definitive Edition':{'Name':'Hitman: Definitive Edition','First release date':'May 15, 2018',
                               'Platfrom':'Xbox One,PS4','Developer':'Io Interactive',
                               'Publisher':'Warner Bros. Interactive Entertainment','Genre':'Action Adventure,','Theme':'',
                               'Franchise':None,'Aliasses':None},
  'Mechs vs. Minions':{'Name':'Mechs vs. Minions','First release date':,
                               'Platfrom':'','Developer':'Riot Games Inc.,Stone Librande',
                               'Publisher':'Riot Games Inc.','Genre':'Tabletop game','Theme':None,
                               'Franchise':'League of Legends','Aliasses':None},
  'European Ship Simulator':{'Name':'','First release date':,
                               'Platfrom':'','Developer':,
                               'Publisher':'','Genre':'','Theme':'',
                               'Franchise':'','Aliasses':},
  'Puzzler: Brain Games':{'Name':'','First release date':,
                               'Platfrom':'','Developer':,
                               'Publisher':'','Genre':'','Theme':'',
                               'Franchise':'','Aliasses':},
  'Payday 3':{'Name':'','First release date':,
                               'Platfrom':'','Developer':,
                               'Publisher':'','Genre':'','Theme':'',
                               'Franchise':'','Aliasses':},
  "Dr. Kawashima's Devilish Brain Training: Can Yo":{'Name':"Dr. Kawashima's Devilish Brain Training: Can You Stay Focused",'First release date':,
                               'Platfrom':'','Developer':,
                               'Publisher':'','Genre':'','Theme':'',
                               'Franchise':'','Aliasses':},
}

In [307]:
pages = {'Borderlands 2 (PS Vita)':'https://www.giantbomb.com/borderlands-2/3030-36055/',
 'Sonic Boom':'https://www.giantbomb.com/sonic-boom/3030-949/',
 'Pokemon X Version':'https://www.giantbomb.com/pokemon-xy/3030-41088/',
 "Madagascar 3: Europe's Most Wanted":'https://www.giantbomb.com/madagascar-3-the-video-game/3030-37449/',
 'Final Fantasy XV Windows Edition':'https://www.giantbomb.com/final-fantasy-xv/3030-21006/',
 'Dungeons & Dragons: Neverwinter':'https://www.giantbomb.com/neverwinter/3030-32417/',
 'Minecraft: Wii U Edition':'https://www.giantbomb.com/minecraft/3030-30475/',
 'DriveClub Bikes':'https://www.giantbomb.com/driveclub/3030-41693/',
 'Minecraft: PlayStation 3 Edition':'https://www.giantbomb.com/minecraft/3030-30475/',
 'Heavy Fire: The Chosen Few':'https://www.giantbomb.com/heavy-fire-afghanistan/3030-37010/',
 'Myst 3DS':'https://www.giantbomb.com/myst/3030-3970/',
 "Deadlight: Director's Cut":'https://www.giantbomb.com/deadlight/3030-37262/',
 'Sly Cooper Collection':"https://www.giantbomb.com/the-sly-collection/3030-31788/",
 'Minecraft: PS Vita Edition':"https://www.giantbomb.com/minecraft/3030-30475/",
 'Resident Evil: Revelations (console version)':"https://www.giantbomb.com/resident-evil-revelations/3030-31737/",
 'Dead Nation (PS Vita)':"https://www.giantbomb.com/dead-nation/3030-27866/",
 'Disgaea 1 Complete':"https://www.giantbomb.com/disgaea-hour-of-darkness/3030-18540/",
 'Code Name: S.T.E.A.M.':"https://www.giantbomb.com/code-name-s-t-e-a-m/3030-46597/",
 'Resident Evil Zero':"https://www.giantbomb.com/resident-evil-0/3030-10247/"}

In [310]:
driver = webdriver.Chrome(executable_path='/Users/elena.morais/Documents/chromedriver_2')

for title in pages.keys():
    print(title)
    driver.get(pages[title])

    time.sleep(random.randint(2,5))
    get_game_details_2(title,driver,games_dict)
    
save_pickle('giant_bomb_updated_game_info.pkl',games_dict)
    
# games_dict

Borderlands 2 (PS Vita)
Sonic Boom
Pokemon X Version
Madagascar 3: Europe's Most Wanted
Final Fantasy XV Windows Edition
Dungeons & Dragons: Neverwinter
Minecraft: Wii U Edition
DriveClub Bikes
Minecraft: PlayStation 3 Edition
Heavy Fire: The Chosen Few
Myst 3DS
Deadlight: Director's Cut
Sly Cooper Collection
Minecraft: PS Vita Edition
Resident Evil: Revelations (console version)
Dead Nation (PS Vita)
Disgaea 1 Complete
Code Name: S.T.E.A.M.
Resident Evil Zero


'giant_bomb_updated_game_info.pkl is saved'

In [312]:
df = pd.DataFrame.from_dict(games_dict, orient='index')
df = df.reset_index().rename(columns={'index':'vgt_name'})
df['fuzzy_ratio'] = df.apply(get_fuzzy_match, axis=1)